In [1]:
# Import Pandas
import pandas as pd

In [2]:
#Clean and export company rating data to a csv
filepath = "../../Scraping/CSV_files/company_rating.csv"
company_rating = pd.read_csv(filepath)

#Fill NaN value to No Rating
com_rating_df=company_rating.fillna("No Rating")

#Save to CSV
com_rating_df.to_csv('../Resources/company_rating.csv', index=False)


In [3]:
##Clean and export housing data to a csv

#Call First Scraping Housing Data
filepath2 = "../../Scraping/CSV_files/cleaned_combined_csv.csv"
housing = pd.read_csv(filepath2)
housing_df=housing.rename(columns={"Asking Price":"Asking_Price"})


#Remove Duplicates
housing_df.drop_duplicates(subset=['Street'],inplace=True)

housing_df=housing_df.drop(columns=["State", "Zip"])

#Save to CSV
housing_df.to_csv('../Resources/housing.csv', index=False)

In [4]:
city_df=pd.DataFrame({"city":housing_df["City"].unique()})
city_df.to_csv('../Resources/city.csv', index=False)

In [5]:
##Clean and export company salary data to a csv

#Call First Scraping salary Data
filepath3 = "../../Scraping/CSV_files/scrape_indeed_combined.csv"
salary_df = pd.read_csv(filepath3)

#Fill NaN values 
salary_df["location"]=salary_df["location"].fillna(" Los Angeles")
salary_df["salary"]=salary_df["salary"].fillna("Unknown")

#Create empty column
salary_df["city"]=""
salary_df["company_id"]=""


#### Creating Company CSV###

#Capturing company list
com_id=list(salary_df["company"].unique())

#Creating a dataframe
id_df=pd.DataFrame({"company":com_id})
id_df["company_id"]=id_df.index


#Adding missing values to company rating CSV
x=[]
y=[]

for index, row in id_df.iterrows():
    if row['company'] not in list(com_rating_df["company"]):
        x.append(row['company'])
        y.append("Rating Unknown")

dict={'company':x,'star':y}
ad=pd.DataFrame(dict)

com_rating_df=com_rating_df.append(ad, ignore_index=True)

        
#Export to CSV
id_df.to_csv('../Resources/company.csv', index=False)
com_rating_df.to_csv('../Resources/company_rating.csv', index=False)

#########

#Write out city
for index, row in salary_df.iterrows():
    row["city"]=row["location"].split(",")[0]

    try:
        row["company_id"]=int(id_df.loc[id_df["company"]==salary_df.loc[index,"company"],"company_id"])
        
    except (TypeError, IndexError):
        row["company_id"]="City Not Posted"
        

#Drop the city not scraped
for index, row in salary_df.iterrows():
    if row['city'] not in list(city_df["city"]):
        salary_df=salary_df.drop([index])    

#Add Index
salary_df["index"]=salary_df.index
        
#Export to CSV
salary_df.to_csv('../Resources/company_salary.csv', index=False)
